## UAV-Cyber-SIM

In [1]:
import logging
import random

from config import Color
from helpers import clean, setup_logging
from helpers.check_resources import PsutilResourceLogger
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigNovis,
    ConfigQGC,
    Gazebo,
    NoVisualizer,
    Simulator,
)

clean()

logger = PsutilResourceLogger(logfile="usage.log", interval=0.01, include_children=True)


setup_logging("main")
logging.info("Starting simulation...")


10:19:49 - main - INFO - Starting simulation...


## Create Plans

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=90) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

gcs=[Color.RED,
    Color.ORANGE,
    Color.GREEN,
    Color.BLUE
    ]
n_uavs_per_gcs = 60
side_len = 10
altitude = 5
max_delay = 3 #sec

base_homes = ENUPose.list([
    (i * 50, j * 3 * side_len, 0, 0)
    for i in range(len(gcs))
    for j in range(n_uavs_per_gcs)
])

base_paths = [
    Plan.create_square_path(side_len=side_len, alt=altitude,heading=0) 
    for _ in base_homes
    ]

colors = [color for color in gcs for _ in range(n_uavs_per_gcs)]

msn_delays=[random.randint(0, max_delay) for _ in base_homes]


## Assign vehicles to GCS (by color)
gcs_names = [f"{color.name} {color.emoji}" for color in gcs]
gcs_sysids = [
    list(range(i * n_uavs_per_gcs + 1, (i + 1) * n_uavs_per_gcs + 1)) 
    for i in range(len(gcs))
    ]

## Gazebo Congiguration

In [3]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/runway3.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
#gaz_config.show()

## QGroundControl Configuration

In [4]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color,delay in zip(base_paths,base_homes,colors,msn_delays):
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay)
qgc_config.show()

# No Visualizer

In [5]:
novis_config = ConfigNovis(origin=gra_origin)
for home in base_homes:
    novis_config.add(base_home=home)

## Visualization Parameters

In [6]:
novis=NoVisualizer(novis_config)

gaz= Gazebo(gaz_config,gra_origin)

qgc=QGC(qgc_config)

## Launch Simulator

In [ ]:
with logger.running():
	simulator = Simulator(
		visualizers=[novis],
		gcs_names=gcs_names,
		gcs_sysids=gcs_sysids,
		missions=[veh.mission for veh in qgc_config.vehicles],
		terminals=['gcs'],
		verbose=1,
	)

	orac = simulator.launch()

10:19:50 - main - INFO - PID 2452816 | CPU  98.0% | RSS   153.64 MB | USS   129.38 MB
10:19:50 - main - INFO - 🙈 Running without visualization.
10:19:51 - main - INFO - 🚀 GCS RED 🟥 launched (PID 2452895)
10:19:51 - main - INFO - 🚀 GCS ORANGE 🟧 launched (PID 2452896)
10:19:51 - main - INFO - 🚀 GCS GREEN 🟩 launched (PID 2452897)
10:19:51 - main - INFO - 🚀 GCS BLUE 🟦 launched (PID 2452898)
10:19:51 - main - INFO - ⏳ Waiting for GCS processes to launch vehicles...
10:19:51 - main - INFO - PID 2452816 | CPU 158.7% | RSS   165.40 MB | USS   131.56 MB
10:19:51 - main - INFO - PID 2452816 | CPU  89.7% | RSS   176.62 MB | USS   133.07 MB
xterm: xterm: Xt error: Can't open display: localhost:10.0
Xt error: Can't open display: localhost:10.0
xterm: Xt error: Can't open display: localhost:10.0
xterm: Xt error: Can't open display: localhost:10.0
10:19:51 - main - INFO - PID 2452816 | CPU  87.5% | RSS   154.67 MB | USS   130.63 MB
10:19:51 - main - INFO - PID 2452816 | CPU  84.9% | RSS   154.67 MB |

10:19:51 - main - INFO - PID 2452816 | CPU  53.3% | RSS   154.67 MB | USS   130.66 MB
10:19:51 - main - INFO - PID 2452816 | CPU  83.5% | RSS   154.67 MB | USS   130.66 MB
10:19:51 - main - INFO - PID 2452816 | CPU  58.3% | RSS   154.67 MB | USS   130.66 MB
10:19:51 - main - INFO - PID 2452816 | CPU  89.4% | RSS   154.67 MB | USS   130.66 MB
10:19:51 - main - INFO - PID 2452816 | CPU  56.7% | RSS   154.67 MB | USS   130.67 MB
10:19:51 - main - INFO - PID 2452816 | CPU  86.1% | RSS   154.67 MB | USS   130.67 MB
10:19:51 - main - INFO - PID 2452816 | CPU  59.7% | RSS   154.67 MB | USS   130.67 MB
10:19:51 - main - INFO - PID 2452816 | CPU  56.1% | RSS   154.67 MB | USS   130.67 MB
10:19:51 - main - INFO - PID 2452816 | CPU 112.1% | RSS   154.67 MB | USS   130.67 MB
10:19:51 - main - INFO - PID 2452816 | CPU  28.5% | RSS   154.67 MB | USS   130.67 MB
10:19:51 - main - INFO - PID 2452816 | CPU  88.6% | RSS   154.67 MB | USS   130.68 MB
10:19:51 - main - INFO - PID 2452816 | CPU  84.7% | RS

## Oracle checking

In [ ]:
with logger.running():
    logging.info("Starting Oracle monitoring...")
    orac.run()
    logging.info("🎉 Oracle monitoring completed!")
    logging.info("🎉 All UAVs have completed their missions!")
    logging.info("🎯 Main simulation process terminating...")

23:31:18 - main - INFO - Starting Oracle monitoring...
23:31:18 - main - INFO - Oracle ⚪: 🏁 Starting Oracle with 240 vehicles
23:31:18 - main - INFO - Oracle ⚪: Monitoring 4 GCS processes
23:31:18 - main - INFO - Oracle ⚪: Entering main monitoring loop...
23:32:40 - main - INFO - Oracle ⚪: Received message 'DONE' from GCS 4
23:32:40 - main - INFO - Oracle ⚪: ✅ Processing DONE from GCS 4
23:32:40 - main - INFO - Oracle ⚪: Removing GCS 4 and its UAVs: [181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240]
23:32:40 - main - INFO - Oracle ⚪: Removing UAV 181 from tracking
23:32:40 - main - INFO - Oracle ⚪: Removing UAV 182 from tracking
23:32:40 - main - INFO - Oracle ⚪: Removing UAV 183 from tracking
23:32:40 - main - INFO - Oracle ⚪: Removing UAV